## DAE_EMT_VS_RLC1

### Import dpsimpy

In [ ]:
import dpsimpy

### Parameters

In [ ]:
resistance = 10;
inductance = 1e-3;
capacitance = 5e-6;

### Powerflow for Initialization

In [ ]:
sim_name_pf = 'DAE_VS_RLC1_PF'
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1_pf')
n2_pf  = dpsimpy.sp.SimNode('n2_pf')
n3_pf  = dpsimpy.sp.SimNode('n3_pf')

# Components
vs_pf = dpsimpy.sp.ph1.VoltageSource('vs_pf', dpsimpy.LogLevel.debug)
vs_pf.set_parameters(V_ref=complex(10,0), f_src=50)
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf', dpsimpy.LogLevel.debug)
r1_pf.set_parameters(R=resistance)
c1_pf = dpsimpy.sp.ph1.Capacitor('c1_pf', dpsimpy.LogLevel.debug)
c1_pf.set_parameters(C=capacitance)
l1_pf = dpsimpy.sp.ph1.Inductor('l1_pf', dpsimpy.LogLevel.debug)
l1_pf.set_parameters(L=inductance)

# Connections
vs_pf.connect([gnd_pf, n1_pf])
r1_pf.connect([n2_pf, n1_pf])
l1_pf.connect([n3_pf, n2_pf])
c1_pf.connect([gnd_pf, n3_pf])

# Define system topology
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf, n3_pf], [vs_pf, r1_pf, l1_pf, c1_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)
logger_pf.log_attribute('n3.v', 'v', n3_pf)
logger_pf.log_attribute('r1.i_intf', 'i_intf', r1_pf)
logger_pf.log_attribute('l1.i_intf', 'i_intf', l1_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### DPsim EMT

In [ ]:
### DPsim EMT simulation
name = 'DAE_Ph3_VS_RLC1'
dpsimpy.Logger.set_log_dir("logs/" + name)
timeStep = 0.0001;

### absolute tolerances
voltage_tol = 1e-8
current_tol = 1e-6
    
### Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC) 
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0]))
n1.dae_set_abs_tolerance(voltage_tol)
n2 = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0]))
n2.dae_set_abs_tolerance(voltage_tol)
n3 = dpsimpy.emt.SimNode('n3', dpsimpy.PhaseType.ABC) 
n3.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0]))
n3.dae_set_abs_tolerance(voltage_tol)


### Components

# voltage source
vs = dpsimpy.emt.ph3.VoltageSource('vs', dpsimpy.LogLevel.debug)
vs.set_parameters(V_ref=dpsimpy.Math.single_phase_variable_to_three_phase(complex(10,0)), f_src=50)
vs.dae_set_abs_tolerance(voltage_tol)

# resistor
r1 = dpsimpy.emt.ph3.Resistor('r1', dpsimpy.LogLevel.debug)
r1.set_parameters(R=dpsimpy.Math.single_phase_parameter_to_three_phase(resistance))
r1.dae_set_abs_tolerance(voltage_tol)

# capacitor
c1 = dpsimpy.emt.ph3.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.set_parameters(C=dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance))

l1 = dpsimpy.emt.ph3.Inductor('l1', dpsimpy.LogLevel.debug)
l1.set_parameters(L=dpsimpy.Math.single_phase_parameter_to_three_phase(inductance))
l1.dae_set_abs_tolerance(current_tol)


### Connections
vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([n3, n2])
c1.connect([gnd, n3])

### Define system topology
system = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, r1, l1, c1])

### Logging
logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('vs', 'v_intf', vs);
logger.log_attribute('vl', 'v_intf', l1);
logger.log_attribute('vc', 'v_intf', c1);
logger.log_attribute('dvc', 'dv_intf', c1);
logger.log_attribute('vr', 'v_intf', r1);
logger.log_attribute('i', 'i_intf', l1);
logger.log_attribute('di', 'di_intf', l1);

### Simulation
sim = dpsimpy.Simulation(name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_solver(dpsimpy.Solver.DAE)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_min_step_size(1e-6)
sim.set_max_step_size(1e-3)
sim.set_relative_rolerance(1e-4)
sim.set_time_step(timeStep)
sim.set_max_err_test_fails(15)
sim.set_final_time(0.1)
sim.add_logger(logger)
vs.set_initial_current(-sim_pf.get_idobj_attr(r1_pf.name(), 'i_intf').get());    
sim.run()
del sim

### Read Results DPSim

In [ ]:
# read Simulink log file

import matplotlib.pyplot as plt
import villas.dataprocessing.readtools as rt
import os

# read EMT results
file_path = os.getcwd() + "/logs/DAE_Ph3_VS_RLC1/DAE_Ph3_VS_RLC1.csv"
ts_dpsim_emt = rt.read_timeseries_dpsim(file_path)

### Read Modelica Results

In [ ]:
import urllib.request
url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/-/raw/18f44dab4b5193855d0c1712dcb98064257e6aee/Modelica/BasicGrids/VS_RLC1.csv'
local_file = 'Results_DAE_EMT_VS_RLC1_Modelica.csv'
urllib.request.urlretrieve(url, local_file) 
ts_modelica_emt = rt.read_timeseries_dpsim(local_file)

### Plot Results DPSim

In [ ]:
# define region of interest
common_time = 50e-6
roi_start = 0
roi_end = 0.04
roi = range(int(roi_start/common_time), int(roi_end/common_time))

# current 
width = 12
height = 3
fig = plt.figure(figsize=(width, height))
fig.suptitle('Current', fontsize=12)
plt.subplot(1, 2, 1)
plt.plot(ts_dpsim_emt['i_0'].interpolate(common_time).time[roi], ts_dpsim_emt['i_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='i_0 - DPSim')
plt.plot(ts_dpsim_emt['i_1'].interpolate(common_time).time[roi], ts_dpsim_emt['i_1'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='i_1 - DPSim')
plt.plot(ts_dpsim_emt['i_2'].interpolate(common_time).time[roi], ts_dpsim_emt['i_2'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='i_2 - DPSim')
plt.plot(ts_modelica_emt['inductor.i'].interpolate(common_time).time[roi], ts_modelica_emt['inductor.i'].interpolate(common_time).values[roi], '--', label='i - Modelica')
plt.plot(ts_modelica_emt['inductor.i'].interpolate(common_time).time[roi], ts_modelica_emt['inductor.i'].interpolate(common_time).values[roi] - ts_dpsim_emt['i_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')

plt.legend(loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(ts_dpsim_emt['di_0'].interpolate(common_time).time[roi], ts_dpsim_emt['di_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='di_0 - DPSim')
plt.plot(ts_modelica_emt['der(voltageSource.i)'].interpolate(common_time).time[roi], -ts_modelica_emt['der(voltageSource.i)'].interpolate(common_time).values[roi], '--', label='di - Modelica')
plt.plot(ts_modelica_emt['der(voltageSource.i)'].interpolate(common_time).time[roi], -ts_modelica_emt['der(voltageSource.i)'].interpolate(common_time).values[roi] - ts_dpsim_emt['di_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')

# component voltages 
width = 18
height = 3
fig = plt.figure(figsize=(width, height))
fig.suptitle('Voltages', fontsize=12)
plt.subplot(1, 5, 1)
plt.plot(ts_dpsim_emt['vs_0'].interpolate(common_time).time[roi], ts_dpsim_emt['vs_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='vs voltage - DPSim')
plt.plot(ts_modelica_emt['voltageSource.v'].interpolate(common_time).time[roi], ts_modelica_emt['voltageSource.v'].interpolate(common_time).values[roi], '--', label='vs voltage - Modelica')
plt.plot(ts_modelica_emt['voltageSource.v'].interpolate(common_time).time[roi], ts_modelica_emt['voltageSource.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vs_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')
plt.subplot(1, 5, 2)
plt.plot(ts_dpsim_emt['vc_0'].interpolate(common_time).time[roi], ts_dpsim_emt['vc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='vc_0 - DPSim')
plt.plot(ts_modelica_emt['capacitor.v'].interpolate(common_time).time[roi], ts_modelica_emt['capacitor.v'].interpolate(common_time).values[roi], '--', label='vc - Modelica')
plt.plot(ts_modelica_emt['capacitor.v'].interpolate(common_time).time[roi], ts_modelica_emt['capacitor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')
plt.subplot(1, 5, 3)
plt.plot(ts_dpsim_emt['vl_0'].interpolate(common_time).time[roi], ts_dpsim_emt['vl_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='vl_0 - DPSim')
plt.plot(ts_modelica_emt['inductor.v'].interpolate(common_time).time[roi], ts_modelica_emt['inductor.v'].interpolate(common_time).values[roi], '--', label='vl - Modelica')
plt.plot(ts_modelica_emt['inductor.v'].interpolate(common_time).time[roi], ts_modelica_emt['inductor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vl_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')
plt.subplot(1, 5, 4)
plt.plot(ts_dpsim_emt['vr_0'].interpolate(common_time).time[roi], ts_dpsim_emt['vr_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='vr_0 - DPSim')
plt.plot(ts_modelica_emt['resistor.v'].interpolate(common_time).time[roi], ts_modelica_emt['resistor.v'].interpolate(common_time).values[roi], '--', label='vr - Modelica')
plt.plot(ts_modelica_emt['inductor.v'].interpolate(common_time).time[roi], ts_modelica_emt['resistor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vr_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')
plt.subplot(1, 5, 5)
plt.plot(ts_dpsim_emt['dvc_0'].interpolate(common_time).time[roi], ts_dpsim_emt['dvc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='dvc_0 - DPSim')
plt.plot(ts_modelica_emt['der(capacitor.v)'].interpolate(common_time).time[roi], ts_modelica_emt['der(capacitor.v)'].interpolate(common_time).values[roi], '--', label='dvc - Modelica')
plt.plot(ts_modelica_emt['der(capacitor.v)'].interpolate(common_time).time[roi], ts_modelica_emt['der(capacitor.v)'].interpolate(common_time).values[roi] - ts_dpsim_emt['dvc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH, label='difference')
plt.legend(loc='lower right')

### Assert

In [ ]:
#compare voltages
time = ts_dpsim_emt['vs_0'].interpolate(common_time).time[roi]
dif_vs = ts_modelica_emt['voltageSource.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vs_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
dif_vr = ts_modelica_emt['resistor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vr_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
dif_vl = ts_modelica_emt['inductor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vl_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
dif_vc = ts_modelica_emt['capacitor.v'].interpolate(common_time).values[roi] - ts_dpsim_emt['vc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
dif_dvc = ts_modelica_emt['der(capacitor.v)'].interpolate(common_time).values[roi] - ts_dpsim_emt['dvc_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH

#compare current
dif_i = ts_modelica_emt['inductor.i'].interpolate(common_time).values[roi] - ts_dpsim_emt['i_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
dif_di = -ts_modelica_emt['der(voltageSource.i)'].interpolate(common_time).values[roi] - ts_dpsim_emt['di_0'].interpolate(common_time).values[roi] * dpsimpy.PEAK1PH_TO_RMS3PH
threashold_voltage = 0.001
threashold_dvoltage = 1.3
threashold_current = 0.0001
threashold_dcurrent = 0.035

import numpy as np
assert np.all(abs(dif_vs)<threashold_voltage)
assert np.all(abs(dif_vr)<threashold_voltage)
assert np.all(abs(dif_vl)<threashold_voltage)
assert np.all(abs(dif_vc)<threashold_voltage)
assert np.all(abs(dif_dvc)<threashold_dvoltage)
assert np.all(abs(dif_i)<threashold_current)
assert np.all(abs(dif_di)<threashold_dcurrent)
